1. Реализовать простейший динамический массив, поддерживающий добавление элемента в конец массива при помощи метода append(e).
Также должны поддерживаться следующие операции:
•	извлечение элемента по индексу;
•	установка нового значения по индексу;
•	получение текущей длины массива. 

In [22]:
class DynamicArray:
    def __init__(self):
        self.array = []
    
    def append(self, element):
        self.array.append(element)
    
    def get_element(self, index):
        if 0 <= index < len(self.array):
            return self.array[index]
        else:
            return None
    
    def set_element(self, index, value):
        if 0 <= index < len(self.array):
            self.array[index] = value
        else:
            raise IndexError("Index out of range")
    
    def length(self):
        return len(self.array)

dynamic_array = DynamicArray()

dynamic_array.append(10)
dynamic_array.append(20)
dynamic_array.append(30)

print("Current array:", dynamic_array.array)
print("Element at index 1:", dynamic_array.get_element(1))

dynamic_array.set_element(1, 25)
print("Updated array:", dynamic_array.array)
print("Length of the array:", dynamic_array.length())


Current array: [10, 20, 30]
Element at index 1: 20
Updated array: [10, 25, 30]
Length of the array: 3


1.1. Добавить реализацию удаления элемента из любого места массива и поддержку корректного вывода массива при помощи функции print.

In [24]:
class DynamicArray:
    def __init__(self):
        self.array = []
    
    def append(self, element):
        self.array.append(element)
    
    def get_element(self, index):
        if 0 <= index < len(self.array):
            return self.array[index]
        else:
            return None
    
    def set_element(self, index, value):
        if 0 <= index < len(self.array):
            self.array[index] = value
        else:
            raise IndexError("Index out of range")
    
    def remove_element(self, index):
        if 0 <= index < len(self.array):
            del self.array[index]
        else:
            raise IndexError("Index out of range")
    
    def length(self):
        return len(self.array)
    
    def __str__(self):
        return str(self.array)

dynamic_array = DynamicArray()

dynamic_array.append(10)
dynamic_array.append(20)
dynamic_array.append(30)

print("Current array:", dynamic_array)

dynamic_array.remove_element(1)
print("Updated array after removing element at index 1:", dynamic_array)

Current array: [10, 20, 30]
Updated array after removing element at index 1: [10, 30]


2. Реализовать генератор, который возвращает значение поочередно извлекаемое из конца двух очередей (в качестве очереди используется deque из collections). Если очередь из которой извлекается элемент пуста - генератор заканчивает работу.

In [23]:
from collections import deque

def extract_from_queues(queue1, queue2):
    while queue1 or queue2:  # Пока хотя бы одна из очередей не пуста
        if queue1:
            yield queue1.pop()
        if queue2:
            yield queue2.pop()

queue1 = deque([1, 2, 3])
queue2 = deque([4, 5, 6])

gen = extract_from_queues(queue1, queue2)

for value in gen:
    print(value)

3
6
2
5
1
4


3. Реализовать классы с медодом action():
а) Класс Pump - в методе action() извлекает очередное значение из генератора и помещает значение в очередь (очередь передается в конструктор).
b) Класс MultiAction - при вызове метода action() n раз вызывает метод action() класса, переданного в конструкторе. Число n также определяется в конструкторе.
c) Класс MultiPump - в методе action() извлекает очередное значение из генератора и помещает значение в одну из очередей (очереди передается в конструкторе); очереди , в которые помещаются очередные значения, меняются по порядку.

In [19]:
from queue import Queue

class Action:
    def action(self):
        print("Hello world")

class Pump(Action):
    def __init__(self, generator, queue):
        self.generator = generator
        self.queue = queue

    def action(self):
        try:
            value = next(self.generator)
            self.queue.put(value)
        except StopIteration:
            print("Generator exhausted")

def simple_generator():
    yield 1
    yield 2
    yield 3

queue = Queue()
generator = simple_generator()
pump = Pump(generator, queue)

for _ in range(3):
    pump.action()

print("Queue contents:", list(queue.queue))

Queue contents: [1, 2, 3]


In [18]:
class MultiAction:
    def __init__(self, x, n): # x - используется для хранения экземпляра класса, который должен выполнять определенное действие, n - сколько раз нужно выполнить действие 
        self.x = x
        self.n = n

    def action(self): # вызываем функцию action() объекта x n раз.
        for i in range(self.n): 
            self.x.action() 

class Action: # вызываем функцию action(), которая выводит строку 
    def action(): 
        print("Hello world")

multi_action = MultiAction(Action, 3) #передаем класс Action и 3 в качестве аргументов
multi_action.action()

Hello world
Hello world
Hello world


In [20]:
from queue import Queue

class Action:
    def action(self):
        print("Hello world")

class MultiPump(Action):
    def __init__(self, generator, queues):
        self.generator = generator
        self.queues = queues
        self.current_queue_index = 0

    def action(self):
        try:
            value = next(self.generator)
            current_queue = self.queues[self.current_queue_index]
            current_queue.put(value)
            self.current_queue_index = (self.current_queue_index + 1) % len(self.queues)
        except StopIteration:
            print("Generator exhausted")

# Пример использования
def simple_generator():
    yield 1
    yield 2
    yield 3

queue1 = Queue()
queue2 = Queue()
generator = simple_generator()
multi_pump = MultiPump(generator, [queue1, queue2])

for _ in range(3):
    multi_pump.action()

print("Queue 1 contents:", list(queue1.queue))
print("Queue 2 contents:", list(queue2.queue))

Queue 1 contents: [1, 3]
Queue 2 contents: [2]


4. При помощи GenFromQ, Pump реализовать систему опработки сообщений. Сообщения создаются генератором сообщений возвращающим случайным образорм одно из сообщений. Сообщения из трех генераторов закачиваются в три очереди при помощи классов Pump, далее при помощи GenFromQ и Pump объединяются в одну очередь и выводятся на экран (можно реализовать при помощи класса с action и вызываемого при помощи MultiPump).

In [21]:
import random
from queue import Queue

class GenFromQ:
    def __init__(self, queue):
        self.queue = queue

    def action(self):
        return self.queue.get()

class Pump:
    def __init__(self, generator, queue):
        self.generator = generator
        self.queue = queue

    def action(self):
        value = next(self.generator)
        self.queue.put(value)

class MultiPump:
    def __init__(self, actions):
        self.actions = actions

    def action(self):
        for action in self.actions:
            action.action()

def message_generator():
    messages = ["Hello", "Hi", "Hey"]
    while True:
        yield random.choice(messages)

# Создаем очереди и генераторы
queue1 = Queue()
queue2 = Queue()
queue3 = Queue()

gen1 = message_generator()
gen2 = message_generator()
gen3 = message_generator()

# Создаем насосы для каждой очереди
pump1 = Pump(gen1, queue1)
pump2 = Pump(gen2, queue2)
pump3 = Pump(gen3, queue3)

# Создаем объединенную очередь
combined_queue = Queue()

# Создаем экземпляры GenFromQ для каждой из трех очередей
gen_from_q1 = GenFromQ(queue1)
gen_from_q2 = GenFromQ(queue2)
gen_from_q3 = GenFromQ(queue3)

# Создаем MultiPump для объединения сообщений в одну очередь
multi_pump = MultiPump([gen_from_q1, gen_from_q2, gen_from_q3])

# Помещаем сообщения в очереди
pump1.action()
pump2.action()
pump3.action()

# Объединяем сообщения в одну очередь и выводим на экран
multi_pump.action()
print("Combined Queue contents:", list(combined_queue.queue))

Combined Queue contents: []
